# Guide

## Import libraries

In [ ]:
# Standard library imports
import numpy as np
import matplotlib.pyplot as plt
import datetime
import scipy.stats as st
from scipy.optimize import curve_fit

## Definition of parameters and constants
I put them here all in the top, to keep track of them. The parameters will be introduced in the text. 

In [ ]:
# Global constants
c    = 299792458         # m/s, speed of light
kB   = 1.380649e-23      # J/K. Boltzmann constant
amu  = 1.66053904e-27    # kg, molar mass conversion constant
h    = 6.62607015e-34    # J s,  Planck constant
hbar = h/(2*np.pi)
g    = 9.81              # m/s**2, constant of acceleration due to Earth's gravity

# properties of setup
Labs     =  0.005
Lfront   =  0.197
Lhex     =  0.390
Lback    =  0.127 - 0.04           
Llc      =  0.15
Ldetect  =  0.60
Lsp      =  3.50 - Ldetect - Llc - Lback - Lhex - Lfront

L        = [Lfront-Labs, Lhex, Lback, Llc, Ldetect, Lsp]

# Properties for BaF (or SrF)
#mass = (88 + 19) *amu   
mass = (138 + 19) *amu   
B    = 0.21594802         # rotational constant of BaF in cm-1 (Master Thesis Jeroen Maat)
mu   = 3.170*0.0168       # dipole moment in cm-1/(kv/cm)  (debye * conversionfactor)

Starkfit = [[0 for i in range(7)] for j in range(5)]
Starkfit[0] = [1.99832523,  2.78038613,  1.0962849,  11.2629089,   1.6618291,   0.,  0.,  0.,  0.]   # parameters for N=1,M=0 state of BaF 
Starkfit[1] = [1.99297730e+00, -2.30557413e-06,  6.22563075e-04,  1.66160290e+00,  2.42295526e-01,  2.69430173e+00,  4.12591610e-01]    # parametersfor N=1,M=1 state of BaF  
                          

delta    = 1.e-6          # used for taking derivative of field. > 1.e-4 the results start to deviate.
dt       = 1.e-4          # timestep in the guiding() function; depends on the acceleration. 
                          # 1.e-4 is sufficient for an error less than 10 micron after 5 meters of flight. 

J2wavenr = 1/(100*h*c)    # factor to convert energy in cm-1 to SI



r0hex    = 6.e-3          # inner radius hexapole
phi0hex  = 4.501e3 #4.5e3          # voltage applied to hexapole rods (plus and minus -> the voltage difference between adjacent rods is 2*phi0hex)

wavevector =  2*np.pi/860e-9
gamma      =  1/60e-9

x_laser    =  10.e-3      # size laserbeam   
v_Doppler  =  0.1         # minimum velocity corresponding to the Doppler temperature in m/s 

s0         =  4.        #12.  # I/I_s
detuning   =  3.2*gamma   # detuning in Hz

xx0      = [0.66e-3,   0,    0.e-3,  0.,   0.,  0., 184.]
dxx      = [0.33e-3,  4.5e-3, 4.5e-3, 0.e-3, 20., 20., 27.]  

nn = 1000   # number of molecules !100000
nnsim = 10000   # number of molecules in simulation !100000
ni = 100      # number of voltage steps !200



## Functions to determine the acceleration experienced by molecules in the guide
Based on numerical approximation of derivative and on full expression of the electrostatic potential.

The expressions for the electrostatic potential for a quadrupole and hexapole are take from Eq. 24 and 26 of van de Meerakker Chem. Rev. 112 4826 (2012), with a2=2 and a3=3, respectively. 

In [ ]:
def WStark(E,s):
    
   muoverB  = mu/B     
   WStark   = B*(Starkfit[s][0] + (np.sqrt(Starkfit[s][1]**2 + (Starkfit[s][2]*muoverB*E)**2) - Starkfit[s][1]) \
                                - (np.sqrt(Starkfit[s][3]**2 + (Starkfit[s][4]*muoverB*E)**2) - Starkfit[s][3]) \
                                - (np.sqrt(Starkfit[s][5]**2 + (Starkfit[s][6]*muoverB*E)**2) - Starkfit[s][5]) \
                 ) 
   return WStark 

In [ ]:
def Phihex(x,y,phi0hex,r0hex): 
    r = (x**2+y**2)**(1/2)
    if r > 0:
        if y > 0. :
            theta = np.arccos(x/r) - (10/180)*np.pi
        else :
            theta = -np.arccos(x/r) - (10/180)*np.pi

    phihex = (r/r0hex)**3 * (phi0hex * np.cos(3*theta))   # with a3=3
    
    return phihex

def Ehex(x,y,phi0hex,r0hex):    
    Ehex = 1.e-5*np.sqrt(((Phihex(x+delta,y,phi0hex,r0hex)-Phihex(x-delta,y,phi0hex,r0hex))/(2.*delta))**2 \
                        +((Phihex(x,y+delta,phi0hex,r0hex)-Phihex(x,y-delta,phi0hex,r0hex))/(2.*delta))**2)
    
                                  # Gives electric field in cm-1/(kV/cm) (hence the factor 1.e-5)
    return Ehex

def Whex(x,y,phi0hex,r0hex,s):
   E        =  Ehex(x,y,phi0hex,r0hex)  
   Whex     =  WStark(E,s)
   return Whex 

def axhex(x,y,phi0hex,r0hex,s):
    axhex = -(((Whex(x+delta,y,phi0hex,r0hex,s)-Whex(x-delta,y,phi0hex,r0hex,s))/J2wavenr)/(2*delta*mass))
    return axhex

def ayhex(x,y,phi0hex,r0hex,s):
    ayhex = -(((Whex(x,y+delta,phi0hex,r0hex,s)-Whex(x,y-delta,phi0hex,r0hex,s))/J2wavenr)/(2*delta*mass))
    return ayhex      

In [ ]:
# see eq. 7.1 Metcalf and van der Straten with factor 4500/15000 to get proper acceleration according to simulations of Roman
def ascat(vx,fudgefactor,s0,detuning):
    ascat = fudgefactor*((gamma/2)*(s0/(1 + s0 + (2*(detuning*gamma + wavevector*vx)/gamma)**2))*hbar*wavevector  \
                        -(gamma/2)*(s0/(1 + s0 + (2*(detuning*gamma - wavevector*vx)/gamma)**2))*hbar*wavevector)/mass  
    return ascat

In [ ]:
# see eq. 7.1 Metcalf and van der Straten with factor 4500/15000 to get proper acceleration according to simulations of Roman
def fitfunction(vx_list,fudgefactor,s0,detuning):
    fit = np.array([0.0 for j in range(len(vx_list))])
    for j in range(len(vx_list)):
        fit[j] = fudgefactor*((gamma/2)*(s0/(1 + s0 + (2*(detuning*gamma + wavevector*vx_list[j])/gamma)**2))*hbar*wavevector  \
                             -(gamma/2)*(s0/(1 + s0 + (2*(detuning*gamma - wavevector*vx_list[j])/gamma)**2))*hbar*wavevector)/mass 
    return fit

# i.e., ascat and fitfunction are the same, but fitfunction returns a list rather than one acceleration

## Checking scattering force


In [ ]:
velocity_sim_s = []
ascat_sim_s = []
velocity_sim = []
ascat_sim = []
vary = []

with open('S12Gamma1_1.dat') as f:
    for line in f:
        data = line.split()
        
for i in range(int(len(data)/2)):
    velocity_sim_s.append(data[2*i].strip('[(,)]'))
    ascat_sim_s.append(data[2*i+1].strip('[(,)]')) 
  
for i in range(int(len(velocity_sim_s))):
    velocity_sim.append(float(velocity_sim_s[i]))
    ascat_sim.append(float(ascat_sim_s[i]))
    vary.append(1.)


plt.plot(velocity_sim,ascat_sim,"r")
plt.xlabel('velocity x,y (m/s)')
plt.ylabel('acceleration (m/s^2)')
plt.show()
        

In [ ]:
fudgefactor = 1/3
s0 = 12
detuning = 3.2
fit = fitfunction(velocity_sim,fudgefactor,s0,detuning)

plt.plot(velocity_sim,ascat_sim,"or")
plt.plot(velocity_sim,fit,"b")
plt.show()

In [ ]:
from scipy.optimize import leastsq
def residual(vars, vx_list, data, eps_data):
   afit = vars          
   chi  = 0. 
   fit  = fitfunction(velocity_sim,afit[0],afit[1],afit[2])
   for j in range(len(vx_list)):
       chi +=    ((ascat_sim - fit[j])/1.)**2/float(len(vx_list)) 
#   print(np.sqrt(chi))
#   print(vars)
   return ((ascat_sim-fit) / 1.)

afit = [fudgefactor,s0,detuning]
vars = afit
out  = leastsq(residual, vars, args=(velocity_sim, ascat_sim, np.sqrt(vary)))

fudgefactor  = out[0][0]
s0           = out[0][1]
detuning     = out[0][2]

fit = fitfunction(velocity_sim,fudgefactor,s0,detuning)

plt.plot(velocity_sim,ascat_sim,"or")
plt.plot(velocity_sim,fit,"b")
plt.show()

print(fudgefactor,s0,detuning)

In [ ]:
#s0 = 0.

## Starting configurations for the molecules
The following is a collection of functions that allow for the definition of the starting configuration of the molecules. They have to be called for every molecule, hence for the fully determined one(s), you also need to provide the function with the index of the molecule.
Clearly, still work in progress.

In [ ]:
def phasespaceellipse2D(xx0,dxx):
    
    itry = 0
    
    hit = False
    while hit == False and itry < 100:
        xx = np.zeros(7)
        itry += 1
#        print("itry:",itry)
        if itry > 99 :
            print("itry exceeded 100!")
               
        xr = np.sqrt(np.random.uniform())            
        theta = np.random.uniform()*2*np.pi
        
        xx[1] = xx0[1] + 0.5*dxx[1]*xr*np.sin(theta)  # Only x-coordinate
        xx[4] = xx0[4] + 0.5*dxx[4]*xr*np.cos(theta)

        xr = np.sqrt(np.random.uniform())            
        theta = np.random.uniform()*2*np.pi
        
        xx[2] = xx0[2] + 0.5*dxx[2]*xr*np.sin(theta)  
        xx[5] = xx0[5] + 0.5*dxx[5]*xr*np.cos(theta)

        if ((xx[1]-xx0[1])**2 + (xx[2]-xx0[2])**2 < (0.5*dxx[1])**2):
            hit = True
    
    xx[0] = np.random.normal(xx0[0],dxx[0])    
    xx[3] = xx0[3]
    xx[6] = np.random.normal(xx0[6],dxx[6])     
    
    if(np.random.uniform() > 0.):                            # in N=1; 3/5 in m=1 and 2/5 in m=0; (if > 0.4 then s=0)
       s = 0
    else: 
       s = 1      
#    
#        xr = np.sqrt(np.random.uniform())            
#        theta = np.random.uniform()*2*np.pi
#    
#        xx[3] = xx0[3] + 0.5*dxx[3]*xr*np.sin(theta)  # Only z-coordinate
#        xx[6] = xx0[6] + 0.5*dxx[6]*xr*np.cos(theta) 
#            
    
    return [xx,s,hit]

## Ways of travel

In [ ]:
def freeflight(endpoint,xxs,hit):                         #here, xxs is assumed to be 1-D arrays with 7 elems; t,x,y,z,vx,vy,vz

    xxp = np.zeros(7)
    for i in range(0,7): xxp[i] = xxs[i]  
    nsteps = 0    
    while(xxs[3] <= xxp[3] < endpoint and hit == True):
            
        xxp[1] += xxp[4]*(endpoint-xxs[3])/xxp[6]
        xxp[2] += xxp[5]*(endpoint-xxs[3])/xxp[6]
        xxp[3]  = endpoint
            
        xxp[0] += (endpoint-xxs[3])/xxp[6]
        
        nsteps     += 1
    
    return [xxp,hit]     


def hexapole(endpoint,phi0hex,r0hex,xxs,s,hit):               #here, xxs is assumed to be 1-D arrays with 7 elems; t,x,y,z,vx,vy,vz

    xxp = np.zeros(7)
    for i in range(0,7): xxp[i] = xxs[i]
    nsteps = 0   
    while((xxs[3] <= xxp[3] < endpoint) and hit == True):
        
#        if (xxp[1]**2 + xxp[2]**2) <= r0hex**2:
        if -phi0hex < Phihex(xxp[1],xxp[2],phi0hex,r0hex) < phi0hex :
        
            xxp[4] += dt*axhex(xxp[1],xxp[2],phi0hex,r0hex,s)  
            xxp[5] += dt*ayhex(xxp[1],xxp[2],phi0hex,r0hex,s) 
            xxp[6] += 0
            
            xxp[1] += dt*xxp[4]
            xxp[2] += dt*xxp[5]
            xxp[3] += dt*xxp[6]
            
            xxp[0] += dt
            
            nsteps += 1
            
        else:                                               # molecule has hit an electrode. Stop calculation.
            hit = False
            break
    
    return [xxp,hit]     

def lasercooling(endpoint,fudgefactor,s0,detuning,xxs,hit):           #here, xxs is assumed to be 1-D arrays with 7 elems; t,x,y,z,vx,vy,vz
   
    xxp = np.zeros(7)
    for i in range(0,7): xxp[i] = xxs[i]
    nsteps = 0   
    while((xxs[3] <= xxp[3] < endpoint) and hit == True):
        
        if((abs(xxp[1]) <= 0.5*x_laser) and (abs(xxp[4]) > 0.5*v_Doppler)) :
            xxp[4] += dt*ascat(xxp[4],fudgefactor,s0,detuning)     
        if((abs(xxp[2]) <= 0.5*x_laser) and (abs(xxp[5]) > 0.5*v_Doppler)) :
            xxp[5] += dt*ascat(xxp[5],fudgefactor,s0,detuning)
          
        xxp[6] += 0
            
        xxp[1] += dt*xxp[4]
        xxp[2] += dt*xxp[5]
        xxp[3] += dt*xxp[6]
            
        xxp[0] += dt
            
        nsteps += 1
        
    return [xxp,hit]     

In [ ]:
x_laser,v_Doppler,s0,detuning

## Trajectories

In [ ]:
xlist = []
y1list = []
y2list = []
y3list = []

xx     = np.zeros(7)

nn = 100                                        # number of molecules
nj = 21                                         # number of writes in trajectory calculation

time = datetime.datetime.now().time()
print("started at:",time)
for n in range(0,nn):
    tempxlist = []
    tempy1list = []
    tempy2list = []
    tempy3list = []
    [xx,s,hit] = phasespaceellipse2D(xx0,dxx)                                    # make package
    tempxlist.append(xx[3])
    tempy1list.append(xx[1])
    tempy2list.append(xx[4])
    tempy3list.append(0.)
    [xx,hit] = freeflight(L[0], xx, hit)                                         # freeflight
    if hit:
        tempxlist.append(xx[3])
        tempy1list.append(xx[1])
        tempy2list.append(xx[4])
        tempy3list.append(0.)
    if L[1] > 0: 
        for j in range(0,nj):
            [xx,hit] = hexapole(L[0] + L[1]*float(j)/float(nj-1), phi0hex, r0hex, xx, s, hit)         # hexapole
            if hit:
               tempxlist.append(xx[3])
               tempy1list.append(xx[1])
               tempy2list.append(xx[4])
               tempy3list.append(axhex(xx[4],xx[5],phi0hex,r0hex,s))
    tempxlist.append(xx[3])
    tempy1list.append(xx[1])
    tempy2list.append(xx[4])
    tempy3list.append(0.)
    [xx,hit] = freeflight(L[0] + L[1] + L[2], xx, hit)                           # freeflight
    if hit:
        tempxlist.append(xx[3])
        tempy1list.append(xx[1])
        tempy2list.append(xx[4])
        tempy3list.append(0.)
    if L[1] > 0: 
        for j in range(0,nj):
            [xx,hit] = lasercooling(L[0] + L[1] + L[2] +L[3]*float(j)/float(nj-1),fudgefactor,s0,detuning,xx,hit)         # lasercooling
            if hit:
               tempxlist.append(xx[3])
               tempy1list.append(xx[1])
               tempy2list.append(xx[4])
               tempy3list.append(ascat(xx[4],fudgefactor,s0,detuning))
    tempxlist.append(xx[3])
    tempy1list.append(xx[1])
    tempy2list.append(xx[4])
    tempy3list.append(0.)
    [xx,hit] = freeflight(L[0] + L[1] + L[2] + L[3] + L[4], xx, hit)             # freeflight
    if hit:
        tempxlist.append(xx[3])
        tempy1list.append(xx[1])
        tempy2list.append(xx[4])
        tempy3list.append(0.)
    [xx,hit] = freeflight(L[0] + L[1] + L[2] + L[3] + L[4] + L[5], xx, hit)             # freeflight
    if hit:
        tempxlist.append(xx[3])
        tempy1list.append(xx[1])
        tempy2list.append(xx[4])
        tempy3list.append(0.)
    xlist.append(tempxlist)
    y1list.append(tempy1list)
    y2list.append(tempy2list)
    y3list.append(tempy3list)
    if np.mod(n,10)==0: print("n of nn:",n,nn)
        
for n in range(0,nn):
    plt.plot(xlist[n],y1list[n],"b") 
plt.show()  

for n in range(0,nn):
    plt.plot(xlist[n],y2list[n],"b")
plt.show()  

for n in range(0,nn):
    plt.plot(xlist[n],y2list[n],"b")
    plt.xlim(L[0]+L[1], L[0]+L[1]+L[2]+L[3]+L[4])
    plt.ylim(-5,5)
plt.show()    

for n in range(0,nn):
    plt.plot(xlist[n],y3list[n],"b")
plt.show()

for n in range(0,nn):
    plt.plot(xlist[n],y3list[n],"b")
    plt.ylim(-1e7,1e7)
plt.show()    

for n in range(0,nn):
    plt.plot(xlist[n],y3list[n],"b")
    plt.xlim(L[0]+L[1], L[0]+L[1]+L[2]+L[3]+L[4])
    plt.ylim(-2000,2000)
plt.show()    

In [ ]:
for n in range(0,nn):
    if(len(xlist[n])>20):
        plt.plot(xlist[n],y1list[n],"r") 
    else:    
        plt.plot(xlist[n],y1list[n],"ob") 
plt.show()

In [ ]:
phi0hex,fudgefactor,s0,detuning

## Phase space distributions

In [ ]:
x10list = []
x11list = []
x12list = []
x13list = []
x14list = []
x15list = []
x16list = []
x20list = []
x21list = []
x22list = []
x23list = []
x24list = []
x25list = []
x26list = []
x30list = []
x31list = []
x32list = []
x33list = []
x34list = []
x35list = []
x36list = []
x40list = []
x41list = []
x42list = []
x43list = []
x44list = []
x45list = []
x46list = []
x50list = []
x51list = []
x52list = []
x53list = []
x54list = []
x55list = []
x56list = []
x60list = []
x61list = []
x62list = []
x63list = []
x64list = []
x65list = []
x66list = []
x70list = []
x71list = []
x72list = []
x73list = []
x74list = []
x75list = []
x76list = []
slist = []

xx     = np.zeros(7)

nn = 10000                                          # number of molecules

time = datetime.datetime.now().time()
print("started at:",time)
for n in range(0,nn):
    [xx,s,hit] = phasespaceellipse2D(xx0,dxx)                                # make package  
    x10list.append(xx[0])
    x11list.append(xx[1])
    x12list.append(xx[2])
    x13list.append(xx[3])
    x14list.append(xx[4])
    x15list.append(xx[5])
    x16list.append(xx[6])
    slist.append(s)
    [xx,hit] = freeflight(L[0], xx, hit)                                   # freeflight
    if (xx[1]**2 + xx[2]**2) >= (2*r0hex)**2:
        hit = False
    if hit == True:                               # make package
        x20list.append(xx[0])
        x21list.append(xx[1])
        x22list.append(xx[2])
        x23list.append(xx[3])
        x24list.append(xx[4])
        x25list.append(xx[5])
        x26list.append(xx[6])
    if L[1] > 0: 
        [xx,hit] = hexapole(L[0] + L[1], phi0hex, r0hex, xx, s, hit)           # hexapole
        if (xx[1]**2 + xx[2]**2) >= (2*r0hex)**2:
            hit = False
        if hit == True:                   # make package
            x30list.append(xx[0])
            x31list.append(xx[1])
            x32list.append(xx[2])
            x33list.append(xx[3])
            x34list.append(xx[4])
            x35list.append(xx[5])
            x36list.append(xx[6])
    [xx,hit] = freeflight(L[0] + L[1] + L[2], xx, hit)                      # freeflight
    if (xx[1]**2 + xx[2]**2) >= (2*r0hex)**2:
        hit = False
    if hit == True:
        x40list.append(xx[0])
        x41list.append(xx[1])
        x42list.append(xx[2])
        x43list.append(xx[3])
        x44list.append(xx[4])
        x45list.append(xx[5])
        x46list.append(xx[6])
    [xx,hit] = lasercooling(L[0] + L[1] + L[2] + L[3],fudgefactor,s0,detuning,xx,hit)       # lasercooling
    if hit == True:
        x50list.append(xx[0])
        x51list.append(xx[1])
        x52list.append(xx[2])
        x53list.append(xx[3])
        x54list.append(xx[4])
        x55list.append(xx[5])
        x56list.append(xx[6])
    [xx,hit] = freeflight(L[0] + L[1] + L[2] + L[3] + L[4], xx, hit)        # freeflight
    if hit == True:
        x60list.append(xx[0])
        x61list.append(xx[1])
        x62list.append(xx[2])
        x63list.append(xx[3])
        x64list.append(xx[4])
        x65list.append(xx[5])
        x66list.append(xx[6])
    [xx,hit] = freeflight(L[0] + L[1] + L[2] + L[3] + L[4] + L[5], xx, hit)        # freeflight
    if hit == True:
        x70list.append(xx[0])
        x71list.append(xx[1])
        x72list.append(xx[2])
        x73list.append(xx[3])
        x74list.append(xx[4])
        x75list.append(xx[5])
        x76list.append(xx[6])
    if np.mod(n,10000)==0: 
        time = datetime.datetime.now().time()
        print("n of nn:",n,nn,"       now:",time)
    
    
time = datetime.datetime.now().time()
print("finished at:",time)

In [ ]:
# Check state distribution

plt.hist(slist, bins=10) 
plt.ylabel('Probability')
plt.xlabel('state')  
plt.show()

In [ ]:
plt.plot(x41list,x44list,"or")
plt.show()     
plt.plot(x51list,x54list,"ob") 
plt.show()    
plt.plot(x61list,x64list,"og") 
plt.show()    

plt.plot(x41list,x42list,"ob") 
plt.show()    
plt.plot(x51list,x52list,"og") 
plt.show()    
plt.plot(x61list,x62list,"oc") 
plt.show() 

In [ ]:
x = x61list
y = x62list

# Define the borders
deltaX = (max(x) - min(x))/10
deltaY = (max(y) - min(y))/10
xmin = min(x) - deltaX
xmax = max(x) + deltaX
ymin = min(y) - deltaY
ymax = max(y) + deltaY
print(xmin, xmax, ymin, ymax)
# Create meshgrid
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

fig = plt.figure(figsize=(8,8))
ax = fig.gca()
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
cset = ax.contour(xx, yy, f, colors='k')
ax.clabel(cset, inline=1, fontsize=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title('2D Gaussian Kernel density estimation')
plt.show()    

fig = plt.figure(figsize=(8,8))
plt.hist2d(x, y, bins=(50, 50), cmap=plt.cm.jet)
plt.show()

In [ ]:
x = x41list
y = x42list

# Define the borders
deltaX = (max(x) - min(x))/10
deltaY = (max(y) - min(y))/10
xmin = min(x) - deltaX
xmax = max(x) + deltaX
ymin = min(y) - deltaY
ymax = max(y) + deltaY
print(xmin, xmax, ymin, ymax)
# Create meshgrid
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

fig = plt.figure(figsize=(8,8))
ax = fig.gca()
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
cset = ax.contour(xx, yy, f, colors='k')
ax.clabel(cset, inline=1, fontsize=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title('2D Gaussian Kernel density estimation')
plt.show()    

fig = plt.figure(figsize=(8,8))
plt.hist2d(x, y, bins=(50, 50), cmap=plt.cm.jet)
plt.show()

In [ ]:
# fitting spatial distribution (horizontal)

xhist = []
yhist = []


yhistp,xhistp = np.histogram(x41list, bins=50)
for i in range(len(yhistp)-1):
    xhist.append(0.5*(xhistp[i] + xhistp[i+1])) 
    yhist.append(yhistp[i])

xdata = np.asarray(xhist)
ydata = np.asarray(yhist)

p0 = [nn/(100), 0., 3.e-3]
print("start parameters",p0)
  
def Gauss(x, A, x0, sigma):
    y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
    return y
parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)
  
fit_A     = parameters[0]
fit_x0    = parameters[1]
fit_sigma = parameters[2]
  
fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()
plt.show()

print("horizontal, before lasercool section")
print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))

xhist = []
yhist = []

yhistp,xhistp = np.histogram(x61list, bins=50)
for i in range(len(yhistp)-1):
    xhist.append(0.5*(xhistp[i] + xhistp[i+1])) 
    yhist.append(yhistp[i])

xdata = np.asarray(xhist)
ydata = np.asarray(yhist)

p0 = [nn/(100), 0., 3.e-3]
print("start parameters",p0)
  
def Gauss(x, A, x0, sigma):
    y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
    return y
parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)
  
fit_A     = parameters[0]
fit_x0    = parameters[1]
fit_sigma = parameters[2]
  
fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()
plt.show()

print("horizontal")
print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))


# fitting spatial distribution (vertical)

xhist = []
yhist = []

yhistp,xhistp = np.histogram(x42list, bins=50)
for i in range(len(yhistp)-1):
    xhist.append(0.5*(xhistp[i] + xhistp[i+1])) 
    yhist.append(yhistp[i])

xdata = np.asarray(xhist)
ydata = np.asarray(yhist)

p0 = [nn/(100), 0., 3.e-3]
print("start parameters",p0)
  
def Gauss(x, A, x0, sigma):
    y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
    return y
parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)
  
fit_A     = parameters[0]
fit_x0    = parameters[1]
fit_sigma = parameters[2]
  
fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()
plt.show()

print("vertical, before lasercool section")
print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))

xhist = []
yhist = []

yhistp,xhistp = np.histogram(x62list, bins=50)
for i in range(len(yhistp)-1):
    xhist.append(0.5*(xhistp[i] + xhistp[i+1])) 
    yhist.append(yhistp[i])

xdata = np.asarray(xhist)
ydata = np.asarray(yhist)

p0 = [nn/(100), 0., 3.e-3]
print("start parameters",p0)
  
def Gauss(x, A, x0, sigma):
    y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
    return y
parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)
  
fit_A     = parameters[0]
fit_x0    = parameters[1]
fit_sigma = parameters[2]
  
fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()
plt.show()

print("vertical")
print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))


In [ ]:
phi0hexlist = []
widthhorlclist = []
widthvertlclist = []
widthhordetlist = []
widthvertdetlist = []
widthhorlist = []
widthvertlist = []
nrmoleculestruedetlist = []
nrmoleculesinzonedetlist = []
nrmoleculestruelist = []
nrmoleculesinzonelist = []

for i in range(0,ni):
    phi0hex =  0.001 +  (float(i)/float(ni))*10000
    phi0hexlist.append(phi0hex)
    
    print("i of ni:",i,ni)
    print("Voltage on hexapole:",phi0hex)
    
    nrmoleculestruedet = 0
    nrmoleculesinzonedet = 0
    nrmoleculestrue = 0
    nrmoleculesinzone = 0
    
    x10list = []
    x11list = []
    x12list = []
    x13list = []
    x14list = []
    x15list = []
    x16list = []
    x20list = []
    x21list = []
    x22list = []
    x23list = []
    x24list = []
    x25list = []
    x26list = []
    x30list = []
    x31list = []
    x32list = []
    x33list = []
    x34list = []
    x35list = []
    x36list = []
    x40list = []
    x41list = []
    x42list = []
    x43list = []
    x44list = []
    x45list = []
    x46list = []
    x50list = []
    x51list = []
    x52list = []
    x53list = []
    x54list = []
    x55list = []
    x56list = []
    x60list = []
    x61list = []
    x62list = []
    x63list = []
    x64list = []
    x65list = []
    x66list = []
    x70list = []
    x71list = []
    x72list = []
    x73list = []
    x74list = []
    x75list = []
    x76list = []
    x1acclist = []
    x4acclist = []

    xx     = np.zeros(7)

    time = datetime.datetime.now().time()
    print("started at:",time)
    for k in range(0,nnsim):
        [xx,s,hit] = phasespaceellipse2D(xx0,dxx)                           # make package      
        x10list.append(xx[0])
        x11list.append(xx[1])
        x12list.append(xx[2])
        x13list.append(xx[3])
        x14list.append(xx[4])
        x15list.append(xx[5])
        x16list.append(xx[6])
        [xx,hit] = freeflight(L[0], xx, hit)                                   # freeflight
        if (xx[1]**2 + xx[2]**2) >= (1.5*r0hex)**2:
            hit = False
        if hit == True:                               # make package      
            x20list.append(xx[0])
            x21list.append(xx[1])
            x22list.append(xx[2])
            x23list.append(xx[3])
            x24list.append(xx[4])
            x25list.append(xx[5])
            x26list.append(xx[6])
        if L[1] > 0: 
            [xx,hit] = hexapole(L[0] + L[1], phi0hex, r0hex, xx, s, hit)           # hexapole
            if (xx[1]**2 + xx[2]**2) >= (1.5*r0hex)**2:
                hit = False
            if hit == True:                   # make package      
                x30list.append(xx[0])
                x31list.append(xx[1])
                x32list.append(xx[2])
                x33list.append(xx[3])
                x34list.append(xx[4])
                x35list.append(xx[5])
                x36list.append(xx[6])
        [xx,hit] = freeflight(L[0] + L[1] + L[2], xx, hit)                      # freeflight
        if (xx[1]**2 + xx[2]**2) >= (1.5*r0hex)**2:
            hit = False
        if hit == True:
            x40list.append(xx[1])
            x41list.append(xx[1])
            x42list.append(xx[2])
            x43list.append(xx[3])
            x44list.append(xx[4])
            x45list.append(xx[5])
            x46list.append(xx[6])
        [xx,hit] = lasercooling(L[0] + L[1] + L[2] + L[3],fudgefactor,s0,detuning,xx,hit)       # lasercooling
        if hit == True:
            x50list.append(xx[0])
            x51list.append(xx[1])
            x52list.append(xx[2])
            x53list.append(xx[3])
            x54list.append(xx[4])
            x55list.append(xx[5])
            x56list.append(xx[6])
        [xx,hit] = freeflight(L[0] + L[1] + L[2] + L[3] + L[4], xx, hit)        # freeflight
        if hit == True:
            x60list.append(xx[0])
            x61list.append(xx[1])
            x62list.append(xx[2])
            x63list.append(xx[3])
            x64list.append(xx[4])
            x65list.append(xx[5])
            x66list.append(xx[6])     
        if hit == True : 
            nrmoleculestruedet += 1       
        if hit == True and (xx[1]**2 <= (5.e-3)**2) and (xx[2]**2 <= (5.e-3)**2):
            nrmoleculesinzonedet += 1  
        [xx,hit] = freeflight(L[0] + L[1] + L[2] + L[3] + L[4] + L[5], xx, hit)        # freeflight
        if hit == True:
            x70list.append(xx[0])
            x71list.append(xx[1])
            x72list.append(xx[2])
            x73list.append(xx[3])
            x74list.append(xx[4])
            x75list.append(xx[5])
            x76list.append(xx[6])
        if hit == True : 
            nrmoleculestrue += 1       
            
        if hit == True and (xx[1]**2 <= (5.e-3)**2) and (xx[2]**2 <= (5.e-3)**2):
            nrmoleculesinzone += 1  
    
        if np.mod(k,1000)==0: 
            time = datetime.datetime.now().time()
            print("k of nnsim:",k,nnsim,"       now:",time)
    
    nrmoleculestruedetlist.append(nrmoleculestruedet)   
    nrmoleculesinzonedetlist.append(nrmoleculesinzonedet)      
    nrmoleculestruelist.append(nrmoleculestrue)   
    nrmoleculesinzonelist.append(nrmoleculesinzone)   

    time = datetime.datetime.now().time()
    print("finished at:",time)
  

# now fitting 

    x = x41list
    y = x42list

    # Define the borders
    deltaX = (max(x) - min(x))/10
    deltaY = (max(y) - min(y))/10
    xmin = min(x) - deltaX
    xmax = max(x) + deltaX
    ymin = min(y) - deltaY
    ymax = max(y) + deltaY
    print(xmin, xmax, ymin, ymax)
    # Create meshgrid
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)

    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
    ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
    cset = ax.contour(xx, yy, f, colors='k')
    ax.clabel(cset, inline=1, fontsize=10)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title('2D Gaussian Kernel density estimation')
    plt.show()    

    fig = plt.figure(figsize=(8,8))
    plt.hist2d(x, y, bins=(50, 50), cmap=plt.cm.jet)
    plt.show()

    xhist = []
    yhist = []

    yhistp,xhistp = np.histogram(x, bins=50)
    for i in range(len(yhistp)-1):
        xhist.append(0.5*(xhistp[i] + xhistp[i+1])) 
        yhist.append(yhistp[i])

    xdata = np.asarray(xhist)
    ydata = np.asarray(yhist)

    p0 = [nnsim/(100), 0., 3.e-3]
    
    def Gauss(x, A, x0, sigma):
        y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
        return y
    parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)

    fit_A     = parameters[0]
    fit_x0    = parameters[1]
    fit_sigma = parameters[2]

    fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
    plt.plot(xdata, ydata, 'o', label='data')
    plt.plot(xdata, fit_y, '-', label='fit')
    plt.legend()
    plt.show()

    print("horizontal")
    print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))
    
    widthhorlclist.append(abs(fit_sigma))

    xhist = []
    yhist = []

    yhistp,xhistp = np.histogram(y, bins=50)
    for i in range(len(yhistp)):
        xhist.append(0.5*(xhistp[i] + xhistp[i])) 
        yhist.append(yhistp[i])

    xdata = np.asarray(xhist)
    ydata = np.asarray(yhist)

    p0 = [nnsim/(100), 0., 3.e-3]
    
    def Gauss(x, A, x0, sigma):
        y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
        return y
    parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)

    fit_A     = parameters[0]
    fit_x0    = parameters[1]
    fit_sigma = parameters[2]

    fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
    plt.plot(xdata, ydata, 'o', label='data')
    plt.plot(xdata, fit_y, '-', label='fit')
    plt.legend()
    plt.show()

    print("vertical")
    print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))
    
    widthvertlclist.append(abs(fit_sigma))
    
# now fitting 

    x = x61list
    y = x62list

    # Define the borders
    deltaX = (max(x) - min(x))/10
    deltaY = (max(y) - min(y))/10
    xmin = min(x) - deltaX
    xmax = max(x) + deltaX
    ymin = min(y) - deltaY
    ymax = max(y) + deltaY
    print(xmin, xmax, ymin, ymax)
    # Create meshgrid
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)

    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
    ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
    cset = ax.contour(xx, yy, f, colors='k')
    ax.clabel(cset, inline=1, fontsize=10)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title('2D Gaussian Kernel density estimation')
    plt.show()    

    fig = plt.figure(figsize=(8,8))
    plt.hist2d(x, y, bins=(50, 50), cmap=plt.cm.jet)
    plt.show()

    xhist = []
    yhist = []

    yhistp,xhistp = np.histogram(x, bins=50)
    for i in range(len(yhistp)-1):
        xhist.append(0.5*(xhistp[i] + xhistp[i+1])) 
        yhist.append(yhistp[i])

    xdata = np.asarray(xhist)
    ydata = np.asarray(yhist)

    p0 = [nnsim/(100), 0., 3.e-3]
    
    def Gauss(x, A, x0, sigma):
        y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
        return y
    parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)

    fit_A     = parameters[0]
    fit_x0    = parameters[1]
    fit_sigma = parameters[2]

    fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
    plt.plot(xdata, ydata, 'o', label='data')
    plt.plot(xdata, fit_y, '-', label='fit')
    plt.legend()
    plt.show()

    print("horizontal")
    print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))
    
    widthhordetlist.append(abs(fit_sigma))

    xhist = []
    yhist = []

    yhistp,xhistp = np.histogram(y, bins=50)
    for i in range(len(yhistp)):
        xhist.append(0.5*(xhistp[i] + xhistp[i])) 
        yhist.append(yhistp[i])

    xdata = np.asarray(xhist)
    ydata = np.asarray(yhist)

    p0 = [nnsim/(100), 0., 3.e-3]
    
    def Gauss(x, A, x0, sigma):
        y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
        return y
    parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)

    fit_A     = parameters[0]
    fit_x0    = parameters[1]
    fit_sigma = parameters[2]

    fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
    plt.plot(xdata, ydata, 'o', label='data')
    plt.plot(xdata, fit_y, '-', label='fit')
    plt.legend()
    plt.show()

    print("vertical")
    print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))
    
    widthvertdetlist.append(abs(fit_sigma))
    
    
    
# now fitting 

    x = x71list
    y = x72list

    # Define the borders
    deltaX = (max(x) - min(x))/10
    deltaY = (max(y) - min(y))/10
    xmin = min(x) - deltaX
    xmax = max(x) + deltaX
    ymin = min(y) - deltaY
    ymax = max(y) + deltaY
    print(xmin, xmax, ymin, ymax)
    # Create meshgrid
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)

    fig = plt.figure(figsize=(8,8))
    ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    cfset = ax.contourf(xx, yy, f, cmap='coolwarm')
    ax.imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
    cset = ax.contour(xx, yy, f, colors='k')
    ax.clabel(cset, inline=1, fontsize=10)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.title('2D Gaussian Kernel density estimation')
    plt.show()    

    fig = plt.figure(figsize=(8,8))
    plt.hist2d(x, y, bins=(50, 50), cmap=plt.cm.jet)
    plt.show()

    xhist = []
    yhist = []

    yhistp,xhistp = np.histogram(x, bins=50)
    for i in range(len(yhistp)-1):
        xhist.append(0.5*(xhistp[i] + xhistp[i+1])) 
        yhist.append(yhistp[i])

    xdata = np.asarray(xhist)
    ydata = np.asarray(yhist)

    p0 = [nnsim/(100), 0., 3.e-3]
    
    def Gauss(x, A, x0, sigma):
        y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
        return y
    parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)

    fit_A     = parameters[0]
    fit_x0    = parameters[1]
    fit_sigma = parameters[2]

    fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
    plt.plot(xdata, ydata, 'o', label='data')
    plt.plot(xdata, fit_y, '-', label='fit')
    plt.legend()
    plt.show()

    print("horizontal")
    print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))
    
    widthhorlist.append(abs(fit_sigma))

    xhist = []
    yhist = []

    yhistp,xhistp = np.histogram(y, bins=50)
    for i in range(len(yhistp)):
        xhist.append(0.5*(xhistp[i] + xhistp[i])) 
        yhist.append(yhistp[i])

    xdata = np.asarray(xhist)
    ydata = np.asarray(yhist)

    p0 = [nnsim/(100), 0., 3.e-3]
    
    def Gauss(x, A, x0, sigma):
        y = A*np.exp(-0.5*(x-x0)**2/sigma**2)
        return y
    parameters, covariance = curve_fit(Gauss, xdata, ydata, p0)

    fit_A     = parameters[0]
    fit_x0    = parameters[1]
    fit_sigma = parameters[2]

    fit_y = Gauss(xdata,fit_A,fit_x0,fit_sigma)
    plt.plot(xdata, ydata, 'o', label='data')
    plt.plot(xdata, fit_y, '-', label='fit')
    plt.legend()
    plt.show()

    print("vertical")
    print("fit values: A,x0,sigma",fit_A,fit_x0,abs(fit_sigma))
    
    widthvertlist.append(abs(fit_sigma))

In [ ]:
plt.plot(phi0hexlist, nrmoleculesinzonedetlist, 'b')
plt.plot(phi0hexlist, nrmoleculestruedetlist, 'g')
plt.plot(phi0hexlist, nrmoleculesinzonelist, 'r')
plt.plot(phi0hexlist, nrmoleculestruelist, 'g')
plt.show()

plt.plot(phi0hexlist, widthhorlclist, 'b')
plt.plot(phi0hexlist, widthvertlclist, 'b')
plt.plot(phi0hexlist, widthhordetlist, 'r')
plt.plot(phi0hexlist, widthvertdetlist, 'r')
plt.plot(phi0hexlist, widthhorlist, 'g')
plt.plot(phi0hexlist, widthvertlist, 'g')
plt.show()

In [ ]:
print(phi0hexlist)

print(nrmoleculesinzonedetlist)
#print(nrmoleculestruedetlist)
print(nrmoleculesinzonelist)
print(nrmoleculestruelist)

#print(widthhorlclist)
#print(widthvertlclist)
#print(widthhordetlist)
#print(widthvertdetlist)
#print(widthhorlist)
#print(widthvertlist)

print(nnsim)

In [ ]:
phi0hexlist_s0eq0 = [0.001, 100.001, 200.001, 300.001, 400.001, 500.001, 600.001, 700.0010000000001, 800.001, 900.001, 1000.001, 1100.001, 1200.001, 1300.001, 1400.0010000000002, 1500.001, 1600.001, 1700.0010000000002, 1800.001, 1900.001, 2000.001, 2100.001, 2200.001, 2300.001, 2400.001, 2500.001, 2600.001, 2700.001, 2800.0010000000007, 2900.001, 3000.001, 3100.001, 3200.001, 3300.001, 3400.0010000000007, 3500.001, 3600.001, 3700.001, 3800.001, 3900.001, 4000.001, 4100.001, 4200.001, 4300.001, 4400.001, 4500.001, 4600.001, 4700.001, 4800.001, 4900.001, 5000.001, 5100.001, 5200.001, 5300.001, 5400.001, 5500.001, 5600.001000000001, 5700.000999999999, 5800.001, 5900.001, 6000.001, 6100.001, 6200.001, 6300.001, 6400.001, 6500.001, 6600.001, 6700.001, 6800.001000000001, 6900.000999999999, 7000.001, 7100.001, 7200.001, 7300.001, 7400.001, 7500.001, 7600.001, 7700.001, 7800.001, 7900.001, 8000.001, 8100.001000000001, 8200.001, 8300.001, 8400.001, 8500.001, 8600.001, 8700.001, 8800.001, 8900.001, 9000.001, 9100.001, 9200.001, 9300.001, 9400.001, 9500.001, 9600.001, 9700.001, 9800.001, 9900.001]
nrmoleculesinzonedetlist_s0eq0 = [55, 53, 55, 53, 54, 59, 66, 70, 87, 125, 160, 203, 342, 428, 517, 617, 689, 761, 817, 761, 819, 861, 796, 767, 820, 777, 717, 698, 679, 691, 650, 626, 576, 535, 520, 478, 443, 451, 407, 413, 376, 341, 306, 323, 309, 292, 312, 266, 271, 239, 266, 212, 245, 226, 202, 223, 195, 193, 185, 184, 180, 171, 144, 163, 162, 154, 158, 140, 160, 147, 137, 124, 121, 127, 119, 127, 118, 122, 122, 150, 125, 123, 123, 112, 107, 123, 104, 145, 107, 118, 129, 146, 128, 112, 130, 146, 141, 131, 129, 134]
nrmoleculesinzonelist_s0eq0 = [11, 7, 9, 10, 9, 11, 14, 8, 16, 30, 53, 51, 92, 113, 125, 123, 133, 132, 106, 109, 117, 108, 102, 85, 75, 84, 77, 90, 69, 73, 45, 62, 58, 43, 40, 63, 41, 42, 31, 33, 34, 31, 29, 28, 33, 25, 31, 16, 26, 28, 30, 23, 24, 24, 21, 26, 18, 25, 18, 13, 11, 10, 12, 21, 15, 13, 16, 17, 14, 14, 11, 10, 11, 6, 9, 14, 10, 8, 14, 18, 13, 10, 11, 12, 9, 17, 7, 11, 11, 7, 13, 14, 14, 12, 15, 5, 15, 15, 13, 12]
nrmoleculestruelist_s0eq0 = [565, 523, 592, 562, 594, 632, 680, 741, 732, 826, 926, 933, 1046, 1098, 1170, 1244, 1316, 1363, 1447, 1349, 1475, 1559, 1513, 1527, 1578, 1566, 1542, 1558, 1545, 1547, 1565, 1604, 1537, 1525, 1526, 1440, 1497, 1529, 1424, 1381, 1397, 1295, 1356, 1326, 1361, 1306, 1354, 1233, 1326, 1303, 1345, 1201, 1234, 1205, 1212, 1209, 1177, 1201, 1121, 1188, 1091, 1138, 1102, 1115, 1055, 1073, 1064, 1056, 1101, 1104, 1032, 1019, 991, 994, 986, 999, 948, 934, 941, 990, 1017, 1010, 873, 918, 871, 941, 860, 942, 906, 852, 897, 876, 845, 803, 848, 844, 833, 855, 861, 845]

widthhorlclist_s0eq0 = [0.005948539241802445, 0.005542593887203104, 0.005783646638865826, 0.005294385137169143, 0.006584399776648233, 0.005660992658946386, 0.005804641800844013, 0.006125371745411408, 0.005930961606957355, 0.005711169612846393, 0.006004185032237734, 0.00600692420036342, 0.005377454248967159, 0.005477495748467304, 0.005107003617714378, 0.004762593659452901, 0.004104550394852859, 0.004026375424612735, 0.003936013491517766, 0.0036290819133506873, 0.0034115415180068723, 0.003115094332982067, 0.003081657045919782, 0.002821710041583309, 0.002621377221589408, 0.0026559239659468283, 0.002366107787785324, 0.002316051640182334, 0.0022189153994128653, 0.0021940345244237285, 0.0019932613761129998, 0.0018726216298222487, 0.0018034212542744452, 0.0017369123716483668, 0.0017169354515596002, 0.0016557448387017853, 0.00156282426707689, 0.0015231576652519386, 0.0014727742330290406, 0.001552202443886183, 0.001493607741205661, 0.0014586010311994405, 0.0013711650945237744, 0.0014162477746156193, 0.0013637425540240907, 0.0013239638270325044, 0.001374770768111809, 0.001395024810152931, 0.0013413495740076614, 0.0013684185988516156, 0.0013850265987658556, 0.001405488174970323, 0.0013976643280048373, 0.0013718027066634499, 0.0014700714044850883, 0.0013918353235214706, 0.0014113342293538254, 0.0014398941869367284, 0.0013667036138329757, 0.0014200385575657746, 0.0013850611393168917, 0.0014058082996977104, 0.0015885417833352968, 0.0015515514523390163, 0.0015008426669542562, 0.001546735947310131, 0.0015939988953431051, 0.0015849891481044748, 0.0016889685109012804, 0.0016890875213824146, 0.001715571895214864, 0.0016563336189704464, 0.0016584934578930536, 0.0017099671028027435, 0.0017373365574691413, 0.0017347110950713875, 0.001654755869920284, 0.001730672992325316, 0.0017901192787931684, 0.001805078590843018, 0.001801900404236569, 0.001825410421615356, 0.0017490838814008358, 0.0019030577664690446, 0.0018865030058101407, 0.0019173166732864664, 0.001885599090364268, 0.0018233440890437333, 0.001963203205458725, 0.0019862560016080883, 0.0018370305478262385, 0.0018840248017792844, 0.001979580656872146, 0.0019130883671218987, 0.001926083868724831, 0.002000655156372151, 0.001855142159703978, 0.0019247679968639366, 0.001858515173080074, 0.0018935061591272965]
widthhordetlist_s0eq0 = [0.012578888402399943, 0.01154272629166999, 0.011675362970255853, 0.01077501557916045, 0.0127512403289029, 0.01140724063628084, 0.0107460725841808, 0.011084713047343198, 0.009289573296371739, 0.00852857238749702, 0.007643158984926124, 0.007044293938645837, 0.005919452234212931, 0.005647112894573595, 0.005133662293414163, 0.0050566736683863986, 0.004692829263110518, 0.004243521762438633, 0.004264114479121878, 0.004441488578158336, 0.004427457507651074, 0.004306208677166336, 0.00449928287444791, 0.0045310579316700595, 0.004619595188407695, 0.004675049539978982, 0.004973086345894751, 0.0052155409786692674, 0.005228847770751129, 0.005042350133891645, 0.005571839421378288, 0.00583098419178477, 0.00537914604802497, 0.006066647941125916, 0.006650141800965367, 0.006512324130059913, 0.00686452800195642, 0.007117391459634345, 0.0071484196846767455, 0.007473194636830389, 0.007711918331125434, 0.008113461424313555, 0.008658077634259888, 0.008395913056709272, 0.008251602076749392, 0.00916902013520705, 0.008512544298409242, 0.008360483337275897, 0.009420744798525019, 0.009556086375331913, 0.008859194940325447, 0.009534472285324478, 0.009422194070381043, 0.009863019745438288, 0.010003475976224545, 0.009650300216964914, 0.010333428783648246, 0.01030443497298372, 0.010111897121083015, 0.009884436729238564, 0.009714909302615952, 0.010245339288619837, 0.011060292392318908, 0.01059749290609317, 0.01050717712290778, 0.010633025019288755, 0.010568952344764165, 0.010020932703383794, 0.010902435796915577, 0.010835492567926886, 0.010752760228416366, 0.010659458301197834, 0.010676984231894362, 0.010480968719405967, 0.010759342416535657, 0.010764563479450862, 0.01029667346895282, 0.009878105183285685, 0.010479803720358205, 0.010330722827603184, 0.010297046427311156, 0.009960917165874321, 0.010093867905533517, 0.010127031982829627, 0.010341137363452392, 0.010649270207556958, 0.010758325402748063, 0.009671373010704764, 0.010020430288875514, 0.010322764570040093, 0.009587987248486052, 0.009420864229060565, 0.009919798985683312, 0.009623327773044699, 0.009280510881152742, 0.009155223972516125, 0.008839248954147646, 0.009067822710969467, 0.00889190969926825, 0.008973309439642706]
widthhorlist_s0eq0 = [0.029516737875645814, 0.027515515300198398, 0.027842414610222653, 0.026320965607523183, 0.029664791789106998, 0.026157176214681537, 0.024502392191428607, 0.023934378720468916, 0.019750948685828065, 0.017702292355158286, 0.01551695219007218, 0.014333805017870595, 0.012661027923461349, 0.012082360105839544, 0.012329520558751977, 0.01241040001875316, 0.012514690778589854, 0.012298217116194932, 0.01302971635766873, 0.013314692740899338, 0.013931182520800311, 0.012723901664224465, 0.015364887206275048, 0.015920947956020264, 0.016685602358228854, 0.017170209374910074, 0.018513128497242723, 0.01946152396946097, 0.019929065424171888, 0.02029163141844573, 0.0223300303244096, 0.023801745562743483, 0.022861409028878642, 0.0251932734991581, 0.026590977112627607, 0.026246335467748228, 0.027520954292883173, 0.027846440367987002, 0.028220506341031473, 0.02925641693183011, 0.029813006181345716, 0.030873545750058698, 0.03289902322609178, 0.03263428824970503, 0.03185375723633728, 0.03404873559533321, 0.031854682437987995, 0.030967735748354652, 0.03458301979289074, 0.03528498392624738, 0.03279315493413781, 0.03499098905339727, 0.034569031614413395, 0.03589218236480664, 0.03622245246788539, 0.03469429776075183, 0.03641253257620402, 0.03668556852547386, 0.03595282287604636, 0.03487235761076962, 0.03433895332429482, 0.036761577261387644, 0.03802283196212917, 0.036845577517456755, 0.037734884715262894, 0.036605339713324275, 0.036540302390062035, 0.03458882898975799, 0.037312047784838494, 0.03699933266891519, 0.03627255092025892, 0.036237541199254854, 0.03646537006170678, 0.03552340646663583, 0.03651878391395917, 0.036315401948933554, 0.03445640766196044, 0.03350674002635784, 0.03521552768167227, 0.03461094119327476, 0.034364548328772906, 0.03319088313428148, 0.03363161743886758, 0.03343848954225898, 0.03436451863028332, 0.03556291496660262, 0.03514136179879812, 0.032310534746128654, 0.03250766182408947, 0.0336484450705673, 0.0312943839273476, 0.03096272360593418, 0.03225013121257033, 0.03172075899210762, 0.03015898244267553, 0.029632462343546873, 0.02877783865731431, 0.0297899471837064, 0.028780945207270692, 0.029531478341446295]

nnsim_s0eq0 = 10000 

phi0hexlist_s0eq4 = [0.001, 100.001, 200.001, 300.001, 400.001, 500.001, 600.001, 700.0010000000001, 800.001, 900.001, 1000.001, 1100.001, 1200.001, 1300.001, 1400.0010000000002, 1500.001, 1600.001, 1700.0010000000002, 1800.001, 1900.001, 2000.001, 2100.001, 2200.001, 2300.001, 2400.001, 2500.001, 2600.001, 2700.001, 2800.0010000000007, 2900.001, 3000.001, 3100.001, 3200.001, 3300.001, 3400.0010000000007, 3500.001, 3600.001, 3700.001, 3800.001, 3900.001, 4000.001, 4100.001, 4200.001, 4300.001, 4400.001, 4500.001, 4600.001, 4700.001, 4800.001, 4900.001, 5000.001, 5100.001, 5200.001, 5300.001, 5400.001, 5500.001, 5600.001000000001, 5700.000999999999, 5800.001, 5900.001, 6000.001, 6100.001, 6200.001, 6300.001, 6400.001, 6500.001, 6600.001, 6700.001, 6800.001000000001, 6900.000999999999, 7000.001, 7100.001, 7200.001, 7300.001, 7400.001, 7500.001, 7600.001, 7700.001, 7800.001, 7900.001, 8000.001, 8100.001000000001, 8200.001, 8300.001, 8400.001, 8500.001, 8600.001, 8700.001, 8800.001, 8900.001, 9000.001, 9100.001, 9200.001, 9300.001, 9400.001, 9500.001, 9600.001, 9700.001, 9800.001, 9900.001]

nrmoleculesinzonedetlist_s0eq4 = [173, 159, 165, 167, 171, 194, 188, 211, 243, 258, 339, 431, 533, 650, 839, 858, 1018, 1062, 1136, 1268, 1263, 1351, 1435, 1475, 1429, 1496, 1455, 1516, 1464, 1455, 1428, 1449, 1437, 1359, 1399, 1427, 1369, 1274, 1260, 1320, 1280, 1255, 1179, 1205, 1140, 1150, 1095, 1082, 1072, 1063, 957, 961, 948, 960, 891, 948, 876, 865, 844, 840, 868, 818, 841, 751, 778, 778, 745, 734, 734, 717, 725, 710, 680, 730, 694, 676, 701, 684, 686, 637, 648, 677, 624, 650, 636, 650, 681, 621, 636, 639, 625, 610, 635, 636, 577, 604, 619, 590, 640, 650] 
nrmoleculesinzonelist_s0eq4 = [77, 87, 74, 77, 82, 100, 89, 110, 136, 150, 224, 305, 385, 481, 637, 688, 850, 929, 965, 1073, 1091, 1150, 1189, 1217, 1140, 1147, 1093, 1123, 1047, 1002, 939, 919, 852, 789, 769, 766, 686, 626, 616, 608, 577, 548, 478, 517, 457, 485, 449, 405, 439, 384, 354, 352, 346, 366, 315, 333, 316, 281, 305, 279, 301, 287, 265, 242, 268, 237, 229, 230, 224, 207, 253, 259, 222, 241, 217, 206, 252, 261, 231, 209, 231, 237, 215, 224, 217, 216, 236, 193, 244, 231, 234, 216, 225, 255, 224, 224, 224, 224, 255, 251]
nrmoleculestruelist_s0eq4 = [566, 533, 579, 543, 577, 656, 655, 728, 783, 808, 894, 947, 1066, 1125, 1264, 1230, 1352, 1377, 1341, 1451, 1425, 1481, 1573, 1594, 1525, 1579, 1546, 1595, 1550, 1539, 1522, 1544, 1557, 1480, 1506, 1549, 1511, 1392, 1421, 1467, 1446, 1414, 1374, 1410, 1325, 1345, 1319, 1291, 1302, 1308, 1196, 1240, 1212, 1235, 1145, 1197, 1201, 1155, 1137, 1146, 1180, 1141, 1180, 1054, 1087, 1090, 1068, 1045, 1036, 1031, 1051, 1032, 993, 1035, 1007, 1000, 984, 955, 997, 946, 933, 960, 892, 935, 931, 924, 909, 875, 917, 884, 861, 848, 888, 863, 802, 855, 861, 791, 867, 862]

widthhorlclist_s0eq4 = [0.005558979518659081, 0.005652477989222218, 0.006076510056905649, 0.0054157946485097015, 0.005457223747870175, 0.00546093868514458, 0.005905081522407831, 0.0062263989081175405, 0.006310440522488408, 0.006497167818293403, 0.006013247744923412, 0.00632814959388798, 0.005947694475082873, 0.0054077096312521436, 0.004639259586983489, 0.004588651892155918, 0.0043984932305046265, 0.004086268723717144, 0.0038779436862989456, 0.0037616900553783562, 0.0034669011302008407, 0.00331925531570785, 0.0030635611387669617, 0.0028208920530626716, 0.002888296718905299, 0.0026700835950456566, 0.0024141202112180566, 0.0023230488340136124, 0.002155736908663895, 0.002058870843004964, 0.0020040606377295326, 0.001870589687400318, 0.0017549184863938753, 0.0018069750044980844, 0.001639227159020075, 0.0016589117346735083, 0.001649969912443369, 0.0015709025484977814, 0.0014932384441395114, 0.00155749744204999, 0.0014962242082054106, 0.0014045814291376256, 0.0014224764444671501, 0.0013805873851238956, 0.001389775000736953, 0.0013746027312048672, 0.0013682779538630287, 0.001350313616776036, 0.0013539167096351089, 0.001408300758162964, 0.0013327874403798415, 0.001396895092291238, 0.0013748647834339097, 0.0013887874722307988, 0.0013538210190391828, 0.001389128521083315, 0.0014645396978903854, 0.0014518446847358525, 0.0015347032770093732, 0.0013645238740992833, 0.0014378509001139474, 0.001457102766601208, 0.001562923784497646, 0.0014802727238952562, 0.0014443846634668981, 0.0015402499745382145, 0.00157794063293135, 0.0014922491103604143, 0.0016843558520866697, 0.001697627113551999, 0.0015961275885952738, 0.0017214381636493025, 0.0016203720611896622, 0.0016780184352917248, 0.001753673374265447, 0.0016381584169298342, 0.0017702431009793877, 0.0016719389969266445, 0.0017224257138391626, 0.0018226027243011364, 0.0017570371472121415, 0.0018067628630697855, 0.0018890246080950058, 0.0017897835469052232, 0.001856595062133374, 0.0018564680642041325, 0.0018155022924873684, 0.0018354274972872838, 0.00188522633869664, 0.001997312945859943, 0.001884088143879944, 0.00190855729834485, 0.001951618987566621, 0.0019451357180798276, 0.0018749223709125917, 0.0020255723152307606, 0.0019259506180070813, 0.0018948166125926445, 0.001959425868423908, 0.0018332895493803289]
widthhordetlist_s0eq4 = [0.004350961806572463, 0.004431483641292404, 0.0062942400216853345, 0.004598639529715775, 0.004844608401946225, 0.005174708599447932, 0.00630821091248581, 0.006296483571984789, 0.006320701268469188, 0.006203582526244557, 0.005657197448013433, 0.0052952841127601995, 0.004798638503201189, 0.004555606683976778, 0.00404012065456481, 0.003949401817029221, 0.0036752475734370895, 0.0033416537645368054, 0.0031700281994092635, 0.003016690021213338, 0.0028164914371293357, 0.002639511311033229, 0.002437190520132625, 0.00229801798251989, 0.0023821565915677118, 0.002241015199265822, 0.0020499111002426087, 0.0020086542126945073, 0.001977454079107836, 0.001964416924862915, 0.0020398220327426065, 0.0019028856006980234, 0.001984188105662463, 0.002027157090025447, 0.002060532770891841, 0.0020506154521042924, 0.0022630472782833744, 0.00216003293665351, 0.0021139249357937533, 0.0021971318303287985, 0.0022636185571963023, 0.0021872248017184345, 0.0023530467492064765, 0.0022318793558201194, 0.002335188792657203, 0.0023766230763514512, 0.0025047860412694897, 0.0025585008628505085, 0.0025780683080265454, 0.002661158567651341, 0.002561758556629083, 0.0027470880969455123, 0.002753183247748806, 0.002838014369749893, 0.002798326334494841, 0.0027244558477619845, 0.00299825454932252, 0.003067800198060442, 0.0031629580388061163, 0.002958270299398552, 0.0031778662217556344, 0.0031462895611705866, 0.0033581542646471617, 0.0032558977133219677, 0.0032051364578824133, 0.0033445216893197263, 0.0033034774856082016, 0.0033881898979707266, 0.0034897557004004397, 0.0035488731764410884, 0.0033456168782748322, 0.0034464736333361333, 0.003576514853013533, 0.0035176932223937923, 0.003568908685441075, 0.0035034477798919165, 0.00346682059293456, 0.0032264008451399493, 0.003451395652197996, 0.003636042759580443, 0.003432996321170407, 0.0034764794515725725, 0.003668005116732074, 0.003585959015913179, 0.0036076197763624837, 0.0036631264311507016, 0.0034549720334988683, 0.003491751800819074, 0.0036326448258588062, 0.003687013211015272, 0.0035499457943294196, 0.0035378744367473925, 0.003659614146811725, 0.003548542382450494, 0.003452967173658692, 0.003553399935649889, 0.0034584119743960077, 0.003445168893236417, 0.0035063393709304077, 0.003296939880918263]
widthhorlist_s0eq4 = [0.005751078722379714, 0.005402612553672066, 0.005984245843051107, 0.0054733270056426435, 0.005570673924140029, 0.005616495121561923, 0.00565527886782971, 0.005100663621317104, 0.005159632784819717, 0.005884854784419537, 0.005229743552358725, 0.00499614606255693, 0.0045439097314683916, 0.004140884173796443, 0.003866815198327661, 0.003664010159019689, 0.0032421330187454626, 0.0030923038323760884, 0.0031510670314670404, 0.003109498333232473, 0.0030954561983361616, 0.002919910700150847, 0.0030212464562702256, 0.003312198984905926, 0.0033399740143576573, 0.003350988947096956, 0.003392628730402365, 0.0035522643304756697, 0.0036713200077113203, 0.003877310532397071, 0.0038031011575835657, 0.0040687730477004126, 0.004245640576643048, 0.004377146214762894, 0.004455474467784324, 0.004589837734304446, 0.005164766749976397, 0.005081021338184329, 0.005025700325593418, 0.005196192894179627, 0.005459722792587017, 0.005501668765623633, 0.005839538305453291, 0.0052727217405681166, 0.005700238018555435, 0.005728238675588733, 0.006314887560717462, 0.0062441924425609835, 0.0062470530274483975, 0.006620539280411221, 0.006494606736685441, 0.0065814665005008285, 0.006610737004520043, 0.006698960826340588, 0.006461043817301408, 0.0065255734824341315, 0.0069471026077822726, 0.0072136444332914825, 0.007316574424622183, 0.006822027145785942, 0.007150088190857078, 0.006996537485397482, 0.00761261448758195, 0.0072887534735353145, 0.007245375687537879, 0.007380889932844452, 0.00737376285087491, 0.007431362223983402, 0.007597848790560548, 0.0076165353575506365, 0.007357598598830267, 0.007402882026679927, 0.007545753271201638, 0.007384024449337292, 0.007466315872836458, 0.007366343140460794, 0.007160975322278626, 0.0066726370995815664, 0.0070633642962988465, 0.007524293954641618, 0.007046854319357454, 0.007140438722544977, 0.007244033442066811, 0.0071206692273021914, 0.0073588101549622205, 0.007371440588485654, 0.00694052887716562, 0.006876090824938212, 0.0072911156711930524, 0.007232184271242072, 0.0069731238020538915, 0.006920848775988117, 0.007167551935483536, 0.006835826734525576, 0.006783218488266407, 0.006854541245636426, 0.006819727214008014, 0.006576854889378575, 0.006764527664885694, 0.006376232845875113]

nnsim_s0eq4 = 10000 

In [ ]:
norm = nrmoleculesinzonelist_s0eq0[0]
for j in range(len(phi0hexlist_s0eq0)):
    nrmoleculesinzonedetlist_s0eq0[j] = nrmoleculesinzonedetlist_s0eq0[j]/norm
for j in range(len(phi0hexlist_s0eq0)):
    nrmoleculesinzonelist_s0eq0[j] = nrmoleculesinzonelist_s0eq0[j]/norm
for j in range(len(phi0hexlist_s0eq0)):
    nrmoleculestruelist_s0eq0[j] = nrmoleculestruelist_s0eq0[j]/norm
    
for j in range(len(phi0hexlist_s0eq4)):
    nrmoleculesinzonedetlist_s0eq4[j] = nrmoleculesinzonedetlist_s0eq4[j]/norm
for j in range(len(phi0hexlist_s0eq4)):
    nrmoleculesinzonelist_s0eq4[j] = nrmoleculesinzonelist_s0eq4[j]/norm
for j in range(len(phi0hexlist_s0eq4)):
    nrmoleculestruelist_s0eq4[j] = nrmoleculestruelist_s0eq4[j]/norm

In [ ]:
norm = 2.355
for j in range(len(phi0hexlist_s0eq0)):
    widthhorlclist_s0eq0[j] = widthhorlclist_s0eq0[j]*norm
for j in range(len(phi0hexlist_s0eq0)):
    widthhordetlist_s0eq0[j] = widthhordetlist_s0eq0[j]*norm
for j in range(len(phi0hexlist_s0eq0)):
    widthhorlist_s0eq0[j] = widthhorlist_s0eq0[j]*norm
    
for j in range(len(phi0hexlist_s0eq4)):
    widthhorlclist_s0eq4[j] = widthhorlclist_s0eq4[j]*norm
for j in range(len(phi0hexlist_s0eq4)):
    widthhordetlist_s0eq4[j] = widthhordetlist_s0eq4[j]*norm
for j in range(len(phi0hexlist_s0eq4)):
    widthhorlist_s0eq4[j] = widthhorlist_s0eq4[j]*norm

In [ ]:
plt.plot(phi0hexlist_s0eq0, nrmoleculesinzonedetlist_s0eq0, 'b')
plt.plot(phi0hexlist_s0eq0, nrmoleculesinzonelist_s0eq0, 'r')
#plt.plot(phi0hexlist_s0eq0, nrmoleculestruelist_s0eq0, 'g')
plt.show()
plt.plot(phi0hexlist_s0eq4, nrmoleculesinzonedetlist_s0eq4, 'b')
plt.plot(phi0hexlist_s0eq4, nrmoleculesinzonelist_s0eq4, 'r')
#plt.plot(phi0hexlist_s0eq0, nrmoleculestruelist_s0eq0, 'g')
plt.show()

#plt.plot(phi0hexlist_s0eq0, nrmoleculesinzonedetlist_s0eq0, 'b')
plt.plot(phi0hexlist_s0eq0, nrmoleculesinzonelist_s0eq0, 'b')
#plt.plot(phi0hexlist_s0eq0, nrmoleculestruelist_s0eq0, 'g')
#plt.plot(phi0hexlist_s0eq4, nrmoleculesinzonedetlist_s0eq4, 'b')
plt.plot(phi0hexlist_s0eq4, nrmoleculesinzonelist_s0eq4, 'r')
#plt.plot(phi0hexlist_s0eq0, nrmoleculestruelist_s0eq0, 'g')
plt.show()

plt.xlabel('Hexapole voltage')
plt.ylabel('FWHM position spread')
plt.plot(phi0hexlist_s0eq0, widthhorlclist_s0eq0, 'b')
plt.plot(phi0hexlist_s0eq0, widthhordetlist_s0eq0, 'r')
plt.plot(phi0hexlist_s0eq0, widthhorlist_s0eq0, 'g')
plt.show()

plt.xlabel('Hexapole voltage')
plt.ylabel('FWHM position spread')
plt.plot(phi0hexlist_s0eq4, widthhorlclist_s0eq4, 'b')
plt.plot(phi0hexlist_s0eq4, widthhordetlist_s0eq4, 'r')
plt.plot(phi0hexlist_s0eq4, widthhorlist_s0eq4, 'g')
plt.show()


#plt.plot(phi0hexlist_s0eq0, widthhorlclist_s0eq0, 'b')
#plt.plot(phi0hexlist_s0eq4, widthhorlclist_s0eq4, 'b')
#plt.show()

In [ ]:
print('No hex, No LC', nrmoleculesinzonelist_s0eq0[0]) 
print('No hex, LC =4', nrmoleculesinzonelist_s0eq4[0]) 
print('Gain only LC', nrmoleculesinzonelist_s0eq4[0]/nrmoleculesinzonelist_s0eq0[0])
print('Optimal voltage NO LC',phi0hexlist_s0eq0[15])
print('No LC', nrmoleculesinzonelist_s0eq0[15]) 
print('Gain only Hex', nrmoleculesinzonelist_s0eq0[15]/nrmoleculesinzonelist_s0eq0[0])
print('Optimal voltage NO LC',phi0hexlist_s0eq4[23])
print('ALL', nrmoleculesinzonelist_s0eq4[23]) 
print('Gain ALL', nrmoleculesinzonelist_s0eq4[23]/nrmoleculesinzonelist_s0eq0[0])